In [5]:
import pandas as pd
import numpy as np

In [27]:
sentiment_daily = pd.concat(
    map(pd.read_csv, ['Dataset/sentiment2018.csv', 'Dataset/sentiment2019p1.csv', 'Dataset/sentiment2019p2.csv', 'Dataset/sentiment2020.csv']), ignore_index=True).drop(columns='Unnamed: 0')

In [28]:
sentiment_daily['Date'] = pd.to_datetime(sentiment_daily['Date'])

sentiment_daily

,Headline,Date,Stock,Neutral,Positive,Negative
0,50 biggest movers from yesterday,2018-11-21,A,8.389207e-01,3.987503e-02,1.212042e-01
1,36 stocks moving in tuesdays midday session,2018-11-20,A,9.999959e-01,1.325494e-07,3.961927e-06
2,shares of agilent technologies are trading up ...,2018-11-20,A,4.851770e-08,1.000000e+00,1.385383e-09
3,morgan stanley maintains overweight on agilent...,2018-11-20,A,3.539342e-07,9.999996e-01,2.367040e-08
4,26 stocks moving in tuesdays premarket session,2018-11-20,A,9.999979e-01,1.054393e-07,2.009748e-06
...,...,...,...,...,...,...
111174,raymond james initiates coverage on zoetis wit...,2020-01-10,ZTS,9.981593e-01,1.731254e-03,1.093797e-04
111175,the daily biotech pulse positive data readouts...,2020-01-10,ZTS,3.736642e-08,1.000000e+00,2.185425e-08
111176,stocks that hit 52week highs on thursday,2020-01-09,ZTS,9.981936e-01,1.231807e-05,1.794028e-03
111177,the daily biotech pulse mercks keytruda snags ...,2020-01-09,ZTS,2.264626e-07,9.999998e-01,3.587590e-08


In [29]:
sentiment_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111179 entries, 0 to 111178
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   Headline  111179 non-null  object        
 1   Date      111179 non-null  datetime64[ns]
 2   Stock     111179 non-null  object        
 3   Neutral   111179 non-null  float64       
 4   Positive  111179 non-null  float64       
 5   Negative  111179 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 5.1+ MB


In [33]:
sentiment_daily['Year'] = sentiment_daily['Date'].dt.year
sentiment_daily['Month'] = sentiment_daily['Date'].dt.month

sentiment_daily.sort_values(by=['Stock', 'Year', 'Month'], inplace=True)

sentiment_daily['pos_ma'] = sentiment_daily.groupby(['Stock', 'Year', 'Month'])['Positive'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())
sentiment_daily['neu_ma'] = sentiment_daily.groupby(['Stock', 'Year', 'Month'])['Neutral'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())
sentiment_daily['neg_ma'] = sentiment_daily.groupby(['Stock', 'Year', 'Month'])['Negative'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())

max_idx = sentiment_daily.groupby(['Stock', 'Year', 'Month'])['Date'].idxmax()
result = sentiment_daily.loc[max_idx]
result['Date']=result['Date'].dt.to_period('M').dt.to_timestamp('M')

In [34]:
result

,Headline,Date,Stock,Neutral,Positive,Negative,Year,Month,pos_ma,neu_ma,neg_ma
65,agilent reports scientific partnership with th...,2018-01-31,A,9.999825e-01,1.966247e-06,1.559735e-05,2018,1,1.966247e-06,9.999825e-01,1.559735e-05
59,bank of america out positive on agilent raises...,2018-02-28,A,4.568109e-09,1.000000e+00,2.321541e-09,2018,2,1.000000e+00,4.568109e-09,2.321541e-09
58,agilent technologies to acquire advanced analy...,2018-03-31,A,9.997074e-01,2.514758e-04,4.108883e-05,2018,3,2.514758e-04,9.997074e-01,4.108883e-05
53,agilent technologies files second civil action...,2018-04-30,A,8.456950e-01,3.495512e-03,1.508095e-01,2018,4,3.495512e-03,8.456950e-01,1.508095e-01
35,agilent to acquire assets of ultra scientific ...,2018-05-31,A,9.999504e-01,9.460290e-07,4.865762e-05,2018,5,9.460290e-07,9.999504e-01,4.865762e-05
...,...,...,...,...,...,...,...,...,...,...,...
111145,zoetis receives fda approval for simparica tri...,2020-02-29,ZTS,3.555827e-05,9.999622e-01,2.226072e-06,2020,2,9.999622e-01,3.555827e-05,2.226072e-06
111135,shares of several healthcare companies are tra...,2020-03-31,ZTS,4.338563e-06,9.999932e-01,2.460831e-06,2020,3,9.999932e-01,4.338563e-06,2.460831e-06
111127,three highquality companies to bet on in a pos...,2020-04-30,ZTS,4.319023e-02,9.563412e-01,4.685563e-04,2020,4,9.563412e-01,4.319023e-02,4.685563e-04
111117,ubs maintains neutral on zoetis lowers price t...,2020-05-31,ZTS,4.827297e-05,5.763391e-05,9.998940e-01,2020,5,5.763391e-05,4.827297e-05,9.998940e-01


In [36]:
result.to_csv('Dataset/sent_analy_18-20.csv')